## Домашняя работа к 5 уроку "Поиск похожих товаров и пользователей. Гибридные рекомендательные системы"

Выполнила Моисеенкова Полина

In [1]:
import pandas as pd
from srs.metrics import precision, precision_at_k
from srs.utils import prefilter_items
from srs.recommenders import MainRecommender

c:\Users\Полина\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv('../retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
item_features = pd.read_csv('../product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [4]:

n_items_before = data_train['item_id'].nunique()

data_train = prefilter_items(data_train, item_features, take_n_popular=5000)

n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 86865 to 5001


In [5]:
recommend = MainRecommender(data_train)

100%|██████████| 2204/2204 [00:00<00:00, 130004.73it/s]


In [6]:
# Рекомендуем товары среди тех, которые пользователь уже купил
recommend.get_own_recommendations(2)

[819235, 1013155, 857259, 1020581, 1098927]

In [7]:
# Рекомендации через стандартные библиотеки implicit
recommend.get_als_recommendations(2)

[1020581, 943116, 5569186, 1098896, 1001907]

In [8]:
# Рекомендуем товары, похожие на топ N-купленных юзером товаров
recommend.get_similar_items_recommendation(2)

[6773032, 7167962, 6773032, 9837328, 6773032]

In [9]:
# Рекомендуем топ N товаров, среди купленных похожими юзерами
recommend.get_similar_user_recommendation(2)

[1031682, 1079594, 1098927, 1128333, 9884861]